In [1]:
import pandas as pd

In [2]:
file_to_load = "Resources/purchase_data.csv" #create path for csv file

purchase_data = pd.read_csv(file_to_load) #read csv file, save it as variable
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
#player count
player_count = purchase_data["SN"].nunique() # needs .nuique to return int values + it ignores NaN values
player_count = pd.DataFrame({"Player Count":[player_count]}) #create Df, define column name, pull value from selected variable

player_count

,Player Count
0,576


In [6]:
# purchasing analysis (Total)
item_count = purchase_data["Item ID"].nunique() #set variable names, from CSV file, column [], give selected function
average_price = purchase_data["Price"].mean()
purchase_count = purchase_data["Purchase ID"].count()
total_revenue = purchase_data["Price"].sum()

In [11]:
purchasing_summary = pd.DataFrame({"Number of Unique Items":[item_count], #create Df, set keys, set values
                                   "Average Price":[average_price],
                                   "Number of Purchases":[purchase_count],
                                   "Total Revenue" : [total_revenue]})

In [12]:
purchasing_summary[["Average Price","Total Revenue"]] \ #convert old Df
= purchasing_summary[["Average Price", "Total Revenue"]].applymap("${:,.2f}".format) #to new Df with formatting style $

purchasing_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [20]:
#Gender Demographics
sn_unique = purchase_data.drop_duplicates(subset = "SN") #define unique buyers from original CSV under column SN

gender_count = sn_unique["Gender"].value_counts() #from sn_unique, create a gender count (3 options)

gender_percentage = gender_count / sn_unique["Gender"].count() * 100 #formula to create percentages

In [21]:
gender_summary_df = pd.DataFrame({"Total Count": gender_count, #create Df, set keys, set values
                                  "Percentage of Players" : gender_percentage})

gender_summary_df["Percentage of Players"] = gender_summary_df["Percentage of Players"] \
.map("{0:.2f}%".format) #convert Df to proper format, this time %

gender_summary_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [24]:
#Purchasing Analysis (Gender)
purchase_count_gender = purchase_data.groupby("Gender")["Item Name"].count() #need to use .groupby bc we're looking at gender groups vs. indiv buyers
average_price_gender = purchase_data.groupby("Gender")["Price"].mean() 
total_price_gender = purchase_data.groupby("Gender")["Price"].sum()
average_total_gender = total_price_gender / gender_count

In [29]:
purchase_summary_df = pd.DataFrame({"Purchase Count": purchase_count_gender, #create Df, set keys, set values
                                    "Average Purchase Price" : average_price_gender,
                                    "Total Purchase Value": total_price_gender,
                                    "Avg Total Purchase per Person" : average_total_gender})

purchase_summary_df[["Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]] \
= purchase_summary_df[["Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]] \
.applymap("${:.2f}".format) #define columns, convert old Df to new Df with formatting

purchase_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [36]:
purchase_count_user = purchase_data.groupby("SN")["Price"].count()
purchase_price_user = purchase_data.groupby("SN")["Price"].mean()
total_purchase_user = purchase_data.groupby("SN")["Price"].sum()

In [59]:
#Age Demographics
bins = [0,9,14,19,24,29,34,39,100] #bins & age_group essentially the same, even though dif. values, bins gives CPU a guide on \
age_group = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"] #where to place an input, age_group defines the index names for the bins
sn_unique["Age Range"] = pd.cut(sn_unique["Age"], bins, labels = age_group) #.cut used to sort data values into bins, this case finding ages of players

purchase_count_age = sn_unique["Age Range"].value_counts() #set variable to save age values
percentage_player_age = (purchase_count_age / sn_unique["SN"].count()) * 100 #calc percentage using last variable / total players

age_summary_df = pd.DataFrame({"Total Count" : purchase_count_age, #create df, set keys, set values
                               "Percentage of Players": percentage_player_age})

age_summary_df = age_summary_df.sort_index() #sort by defined index values age_group

age_summary_df["Percentage of Players"] = age_summary_df["Percentage of Players"].map("{:.2f}%".format) #convert formatting style

age_summary_df



<ipython-input-59-4fe29e283863>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sn_unique["Age Range"] = pd.cut(sn_unique["Age"], bins, labels = age_group)


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [72]:
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], bins, labels = age_group) #sort ages into bins
purchase_age_count = purchase_data["Age Ranges"].value_counts()

average_price = purchase_data.groupby("Age Ranges")["Price"].mean()
total_purchase_age = purchase_data.groupby("Age Ranges")["Price"].sum()
average_purchase_age = total_purchase_age / purchase_age_count

purchase_summary_age = pd.DataFrame({"Purchase Count" : purchase_age_count, #create df, set keys, set values
                                     "Average Purchase Price" : average_price,
                                     "Total Purchase Value" : total_purchase_age,
                                     "Avg Total Purchase per Person" : average_purchase_age})

purchase_summary_age[["Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]] \
= purchase_summary_age[["Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]] \
.applymap("${:.2f}".format) #convert formatting style

purchase_summary_age = purchase_summary_age.rename_axis("Age Ranges") #need to add title to index for it to make sense to viewer

purchase_summary_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,$1114.06,$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


In [38]:
#Top Spenders
user_summary = pd.DataFrame({"Purchase Count": purchase_count_user, #create df, add keys, add values
                             "Average Purchase Price" : purchase_price_user,
                             "Total Purchase Value" : total_purchase_user})

user_summary = user_summary.sort_values(by = "Total Purchase Value", ascending = False) #sort SN by highest total purchase value

user_summary[["Average Purchase Price", "Total Purchase Value"]] \
= user_summary[["Average Purchase Price", "Total Purchase Value"]].applymap("${:.2f}".format) #convert formatting style

user_summary.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [42]:
#most popular items
most_purchased_item = purchase_data.groupby(["Item ID", "Item Name"])["Price"].count() #defines purchase count for given product
popular_item = purchase_data.groupby(["Item ID", "Item Name"])["Price"].sum() #calc sum of total purchase value for given product
item_price = popular_item / most_purchased_item #define third variable using math and other 2 variable values

In [43]:
item_summary = pd.DataFrame({"Purchase Count": most_purchased_item, #create df, add keys, add values
                             "Item Price" : item_price,
                             "Total Purchase Value" : popular_item})

item_summary = item_summary.sort_values("Purchase Count", ascending = False) #sort values by highest purchase count
highest_purchase_value = item_summary.sort_values("Total Purchase Value", ascending = False) #sort values by highest total purchase value

item_summary[["Item Price", "Total Purchase Value"]] \
= item_summary[["Item Price", "Total Purchase Value"]].applymap("${:.2f}".format) #convert formatting

item_summary.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [61]:
# most profitable items
highest_purchase_value[["Item Price", "Total Purchase Value"]] \
= highest_purchase_value[["Item Price", "Total Purchase Value"]].applymap("${:.2f}".format) #convert formatting

highest_purchase_value.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
